## DNN
- 딥러닝의 기본형태인 DNN(Deep neural network)을 통해 날씨에 따른 집단별 판매량 및 SNS 언급량을 예측합니다,

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/weather_bigdata_contest/code

/content/drive/MyDrive/weather_bigdata_contest/code


### 1) 데이터 전처리

In [5]:
wea_sale_ohe = pd.read_csv('../data/weather_sale_ohe.csv')
wea_sns_ohe = pd.read_csv('../data/weather_sns_ohe.csv')

In [6]:
def sale_dataset(ohe_df):
    # 상품 목록을 정의합니다. 
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()
    # 상품 및 날짜로 데이터를 정렬합니다.
    ohe_df.sort_values(goods+['날짜'], inplace=True)
    # 정렬로 뒤죽박죽이 된 행 인덱스를 초기화합니다. 
    ohe_df.index = range(0, len(ohe_df))

    # 모델의 성능 향상을 위해 MinMaxScale을 진행합니다.
    sc = MinMaxScaler(feature_range=(0, 1))
    scaled_ohe = sc.fit_transform(ohe_df.loc[:,'평균기온(°C)':'미세먼지(PM10)'].values)

    # 반드시 판매량 데이터를 마지막 열에 두어야 합니다.
    # scale된 데이터를 병합해서 total data를 생성합니다.
    # 상품 소분류 원핫인코딩 열은 MinMaxScale을 진행하지 않았습니다. (0과 1로 이루어졌으므로)
    total = np.hstack([ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].values,
                    scaled_ohe,
                    ohe_df.loc[:,'20대 여성 판매량(개)':'60대 남성 판매량(개)'].values])

    # 가열식 가습기 판매 데이터 개수의 80% 입니다.
    index =int(len(ohe_df[ohe_df['소분류_가열식 가습기']==1]) * 0.8)

    # 가스온수기 판매 데이터의 80%를 train data로, 20%를 test data로 정의합니다.
    train = total[ohe_df['소분류_가열식 가습기']==1][:index]
    test = total[ohe_df['소분류_가열식 가습기']==1][index:]

    # 전체 상품 목록 리스트
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 각 상품의 판매 데이터의 80%는 train, 20%는 test data에 추가합니다. 
    for good in goods[1:]:
        index =int(len(ohe_df[ohe_df[good]==1]) * 0.8)
        train = np.vstack([train, total[ohe_df[good]==1][:index]])
        test = np.vstack([test, total[ohe_df[good]==1][index:]])

    day = 20

    # X_train, y_train 생성

    df = train[train[:, 0]==1] 
    X_train = df[day-1:, :-10]
    y_train = df[day-1:, -10:]

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = train[train[:, good_index]==1] 
      X_train = np.vstack([X_train, df[day-1:, :-10]])
      y_train = np.vstack([y_train, df[day-1:, -10:]])

    df = test[test[:, 0]==1] 
    X_test = df[day-1:, :-10]
    y_test = df[day-1:, -10:]

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = test[test[:, good_index]==1] 

      # 해당 판매 데이터를 day 기준으로 분할합니다.(20일)
      X_test = np.vstack([X_test, df[day-1:, :-10]])
      y_test = np.vstack([y_test, df[day-1:, -10:]])

    return X_train, X_test, y_train, y_test

In [7]:
def sns_dataset(ohe_df):
    # 상품 목록을 정의합니다. 
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 상품 및 날짜로 데이터를 정렬합니다.
    ohe_df.sort_values(goods+['날짜'], inplace=True)

    # 정렬로 뒤죽박죽이 된 행 인덱스를 초기화합니다. 
    ohe_df.index = range(0, len(ohe_df))

    # 모델의 성능 향상을 위해 MinMaxScale을 진행합니다.
    sc = MinMaxScaler(feature_range=(0, 1))
    scaled_ohe = sc.fit_transform(ohe_df.loc[:,'평균기온(°C)':'미세먼지(PM10)'].values)

    # 반드시 판매량 데이터를 마지막 열에 두어야 합니다.
    # scale된 데이터를 병합해서 total data를 생성합니다.
    # 상품 소분류 원핫인코딩 열은 MinMaxScale을 진행하지 않았습니다. (0과 1로 이루어졌으므로)
    total = np.hstack([ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].values,
                    scaled_ohe,
                    ohe_df.loc[:,'SNS언급량'].values.reshape(-1, 1)])

    # 가열식 가습기 판매 데이터 개수의 80% 입니다.
    index =int(len(ohe_df[ohe_df['소분류_가열식 가습기']==1]) * 0.8)

    # 가스온수기 판매 데이터의 80%를 train data로, 20%를 test data로 정의합니다.
    train = total[ohe_df['소분류_가열식 가습기']==1][:index]
    test = total[ohe_df['소분류_가열식 가습기']==1][index:]

    # 전체 상품 목록 리스트
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 각 상품의 판매 데이터의 80%는 train, 20%는 test data에 추가합니다. 
    for good in goods[1:]:
        index =int(len(ohe_df[ohe_df[good]==1]) * 0.8)
        train = np.vstack([train, total[ohe_df[good]==1][:index]])
        test = np.vstack([test, total[ohe_df[good]==1][index:]])

    day = 20

    df = train[train[:, 0]==1] 
    X_train = df[day-1:, :-1]
    y_train = df[day-1:, -1].reshape(-1,1)

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = train[train[:, good_index]==1] 
      X_train = np.vstack([X_train, df[day-1:, :-1]])
      y_train = np.vstack([y_train, df[day-1:, -1].reshape(-1,1)])

    df = test[test[:, 0]==1] 
    X_test = df[day-1:, :-1]
    y_test = df[day-1:, -1].reshape(-1,1)

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = test[test[:, good_index]==1] 

      # 해당 판매 데이터를 day 기준으로 분할합니다.(20일)
      X_test = np.vstack([X_test, df[day-1:, :-1]])
      y_test = np.vstack([y_test, df[day-1:, -1].reshape(-1,1)])

    return X_train, X_test, y_train.reshape(-1), y_test.reshape(-1)

### 2) 연령, 성별 카테고리별 총 판매량 예측
- 날씨에 따른 성별 및 연령별 판매량을 예측합니다.

In [8]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [9]:
with tf.device('/device:GPU:0'):
    dnn = keras.Sequential([
                        keras.layers.Dense(64, input_dim = 134),
                        keras.layers.BatchNormalization(),
                        keras.layers.ReLU(),
                        keras.layers.Dense(64),
                        keras.layers.BatchNormalization(),
                        keras.layers.ReLU(),
                        keras.layers.Dense(10)
])
    adam = keras.optimizers.Adam(learning_rate = 0.01)
    dnn.compile(loss = 'mean_squared_error', optimizer = adam)
    dnn.fit(X_train, y_train, epochs = 300, validation_data = (X_test, y_test), batch_size = 1000)

    pred = dnn.predict(X_test)

    dnn_sale_mse = mean_squared_error(pred, y_test)
    dnn_sale_mae = mean_absolute_error(pred, y_test)
    print('DNN의 판매량 예측 MSE: ', dnn_sale_mse)
    print('DNN의 판매량 예측 RMSE: ', np.sqrt(dnn_sale_mse))
    print('DNN의 판매량 예측 MAE: ', dnn_sale_mae)

Epoch 1/300
70/70 [==============================] - 4s 8ms/step - loss: 1495.8705 - val_loss: 1775.1488
Epoch 2/300
70/70 [==============================] - 0s 6ms/step - loss: 501.5292 - val_loss: 1503.9939
Epoch 3/300
70/70 [==============================] - 0s 6ms/step - loss: 450.8946 - val_loss: 1184.6226
Epoch 4/300
70/70 [==============================] - 0s 6ms/step - loss: 430.0665 - val_loss: 859.7441
Epoch 5/300
70/70 [==============================] - 0s 5ms/step - loss: 425.8440 - val_loss: 492.7516
Epoch 6/300
70/70 [==============================] - 0s 6ms/step - loss: 419.5385 - val_loss: 496.5818
Epoch 7/300
70/70 [==============================] - 0s 6ms/step - loss: 413.0411 - val_loss: 440.4165
Epoch 8/300
70/70 [==============================] - 0s 6ms/step - loss: 412.9690 - val_loss: 515.8122
Epoch 9/300
70/70 [==============================] - 0s 6ms/step - loss: 412.0612 - val_loss: 448.7304
Epoch 10/300
70/70 [==============================] - 0s 6ms/step - l

### 3) SNS 언급량 예측
- 날씨에 따른 SNS 언급량을 예측합니다.

In [10]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [11]:
with tf.device('/device:GPI:0'):
    dnn = keras.Sequential([
                        keras.layers.Dense(64, input_dim = 134),
                        keras.layers.BatchNormalization(),
                        keras.layers.ReLU(),
                        keras.layers.Dense(64),
                        keras.layers.BatchNormalization(),
                        keras.layers.ReLU(),
                        keras.layers.Dense(1)
])

    dnn.compile(loss = 'mean_squared_error', optimizer = 'adam')

    dnn.fit(X_train, y_train, epochs = 300, validation_data = (X_test, y_test), batch_size = 1000)

    pred = dnn.predict(X_test)

    dnn_sns_mse = mean_squared_error(pred, y_test)
    dnn_sns_mae = mean_absolute_error(pred, y_test)
    print('DNN의 SNS 언급량 예측 MSE: ', dnn_sns_mse)
    print('DNN의 SNS 언급량 예측 RMSE: ', np.sqrt(dnn_sns_mse))
    print('DNN의 SNS 언급량 예측 MAE: ', dnn_sns_mae)

Epoch 1/300
72/72 [==============================] - 1s 7ms/step - loss: 36412.9062 - val_loss: 32969.4922
Epoch 2/300
72/72 [==============================] - 0s 5ms/step - loss: 33823.1523 - val_loss: 32122.1953
Epoch 3/300
72/72 [==============================] - 0s 6ms/step - loss: 31607.5859 - val_loss: 31168.7852
Epoch 4/300
72/72 [==============================] - 0s 6ms/step - loss: 29400.2129 - val_loss: 30054.5098
Epoch 5/300
72/72 [==============================] - 0s 5ms/step - loss: 27015.4590 - val_loss: 28389.3379
Epoch 6/300
72/72 [==============================] - 0s 5ms/step - loss: 24411.8184 - val_loss: 25874.3164
Epoch 7/300
72/72 [==============================] - 0s 5ms/step - loss: 21723.2969 - val_loss: 22733.7520
Epoch 8/300
72/72 [==============================] - 0s 5ms/step - loss: 19068.1543 - val_loss: 19068.0254
Epoch 9/300
72/72 [==============================] - 0s 6ms/step - loss: 16462.7285 - val_loss: 14747.4355
Epoch 10/300
72/72 [=================